In [19]:
import numpy as np
import pandas as pd
import re
#my_regex = "\（.*?\）|\(.*?\)+\(\^一二三四五六七八九零十1234567890\)\]"
#my_regex = r"[\(\（]([^一二三四五六七八九零十1234567890]{1}[^\)\）]+|[一二三四五六七八九零十1234567890]{1}[^\)\）]{1,})[\)\）]"

In [20]:
df=pd.read_csv('.//data//Result_Sbert.csv', index_col=0)
#df=pd.read_csv('.//data//Result_R5.csv', index_col=0)
df.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
0,臺灣桃園地方法院,2018-04-18T00:00:00+08:00,"106,易,1093",刑事,妨害公務,判決,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣桃園地方法院刑事判決106年度易字第1093號公訴人臺灣桃園地方法院檢察署檢察官被告邱建...,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...
2,臺灣臺中地方法院,2011-01-25T00:00:00+08:00,"100,中交簡,100",刑事,公共危險,判決,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,臺灣臺中地方法院刑事簡易判決100年度中交簡字第100號聲請人臺灣臺中地方法院檢察署檢察官被...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...


In [21]:
test=  pd.read_csv('.//data//Result_Test.csv', index_col=0)
test.head(2)

,court,date,no,sys,reason,type,mainText,judgement,Situation
id,,,,,,,,,
522,臺灣臺中地方法院,2017-02-06T00:00:00+08:00,"105,中簡,1134",刑事,傷害,判決,易佳騰犯傷害罪，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事簡易判決105年度中簡字第1134號聲請人臺灣臺中地方法院檢察署檢察官被...,易佳騰於民國103年12月30日22時53分許，在臺中市○○區○○街000號，因工程款糾紛與...
901,臺灣臺中地方法院,2017-06-21T00:00:00+08:00,"105,原易,51",刑事,傷害,判決,宋約翰傷害人之身體，累犯，處有期徒刑肆月，如易科罰金，以新臺幣壹仟元折算壹日。,臺灣臺中地方法院刑事判決105年度原易字第51號公訴人臺灣臺中地方法院檢察署檢察官被告宋約翰...,宋約翰前於民國103年間因公共危險案件，經本院以103年度中交簡字第3997號判決判處有期徒...


In [22]:
df1 = df[~df['no'].isin(test['no'])]
df1['reason'] = df1['reason'].str.replace('等','')
df1['reason'] = df1['reason'].str.replace('罪','')
df1['reason'] = df1['reason'].str.replace("條例","")
df1['reason'] = df1['reason'].str.replace("防制","")
df1['reason'] = df1['reason'].str.replace("管制","")
df1['reason'] = df1['reason'].str.replace("違反","")

print("總筆數",df1['reason'].count(), "\n")
df1.loc[:,'reason'].value_counts()

#df1['Situation'] = df1['Situation'].str.replace(my_regex, "")
#ierror = 0
#for index, row in df1.iterrows():
#    if (row.Situation != row.Situation2):
#        print (index, "\n", row.Situation, "\n", row.Situation2)
#        ierror += 1
#        if (ierror > 5):
#            break

總筆數 4054 



c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\user\pycharmprojects\pytorch\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

公共危險      347
毀損        328
妨害公務      320
妨害名譽      308
妨害自由      266
侵占        231
傷害        221
搶奪        220
過失傷害      216
竊盜        204
殺人未遂      202
妨害風化      190
著作權法      190
偽造文書      171
妨害性自主     168
詐欺        158
強盜        141
毒品危害       65
毀棄損壞       38
殺人         37
過失致死        7
貪污          6
賭博          6
槍砲彈藥刀械      5
醫師法         3
商標法         2
藥事法         2
替代役實施       1
偽證          1
Name: reason, dtype: int64

In [23]:
from sentence_transformers import SentenceTransformer, util
embedder=SentenceTransformer('bert-base-chinese')

import pandas as pd
import time
tStart = time.time()

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [24]:
class Data(object):
    def __init__(self):
        self.no = "0"
        self.reason = ""
        self.Situation = ""

In [25]:
#embedder  SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# Corpus with example sentences
#corpus  ['A man is eating food.',
#          'A man is eating a piece of bread.',
#          'The girl is carrying a baby.',
#          'A man is riding a horse.',
#          'A woman is playing violin.',
#          'Two men pushed carts through the woods.',
#          'A man is riding a white horse on an enclosed ground.',
#          'A monkey is playing drums.',
#          'A cheetah is running behind its prey.'
#          ]
corpus = []
corpusList = []
for index, row in df1.iterrows():
    if (row.Situation == None):
        continue
    if (isinstance(row.Situation, float)):
        continue
    if (len(row.Situation) == 0):
        continue
    if (index == 5):
        print()
    d = Data()
    d.no = row.no
    d.reason = row.reason
    d.Situation = row.Situation
    corpusList.append(d)
    corpus.append(row.Situation)
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
print(len(corpus))


4052


In [26]:
tEnd = time.time()#計時結束
#列印結果
print("轉向量時間", "%f sec" % (tEnd - tStart))

轉向量時間 24.466129 sec


In [27]:
# Query sentences:
#queries  ['A man is eating pasta.', 'Someone in a gorilla costume is playing a set of drums.', 'A cheetah chases prey on across a field.']
queries = []
specialtest  ='詎其猶不知悔改，復基於意圖為自己不法所有之竊盜犯意，於102年7月5日上午6時許（起訴書誤載為5時許），見林振億位於高雄市○○區○○街000號之住處大門未上鎖，竟未經允准侵入上開住宅內，徒手竊取行動電話5支、無線電3支、數位相機1臺及存錢筒1個（內有現金約新臺幣【下同】4,000元，上開物品共計價值47,000元），得手後離去；嗣經林振億發現遭竊後報警，為警在現場採得指紋後送鑑定後，結果與邱東成之指紋相符，而查悉上情。'
#specialtest = re.sub(my_regex, "", specialtest)
queries.append(specialtest)
for index, row in test.iterrows():
    queries.append(str(row.Situation))
print(len(queries))

22


In [28]:
# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = 5
i = 0
show = 0

for query in queries:
    tStart2 = time.time()
    query_embedding = embedder.encode(query, convert_to_tensor=True)
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    cos_scores = cos_scores.cpu()


    #We use np.argpartition, to only partially sort the top_k results
    top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]
    print("\n\n\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    no = 1
    for idx in top_results[0:top_k]:
        if (corpus[idx] == specialtest):
            continue
        print(no, "  "
              , corpusList[idx].no,"\t"
              ,  corpusList[idx].reason
              , "\t(Score: %.4f)" % (cos_scores[idx]))
        if (len(corpus[idx].strip()[0:100]) > 100):
            print (corpus[idx].strip()[0:100])
        else:
            print (corpus[idx].strip()[0:100])
        print('\n')
        no = no + 1
        show += 1
    if (show > 5):break

    tEnd2 = time.time()#計時結束
    print("COS相似計算時間", "%f sec" % (tEnd2 - tStart2))






Query: 詎其猶不知悔改，復基於意圖為自己不法所有之竊盜犯意，於102年7月5日上午6時許（起訴書誤載為5時許），見林振億位於高雄市○○區○○街000號之住處大門未上鎖，竟未經允准侵入上開住宅內，徒手竊取行動電話5支、無線電3支、數位相機1臺及存錢筒1個（內有現金約新臺幣【下同】4,000元，上開物品共計價值47,000元），得手後離去；嗣經林振億發現遭竊後報警，為警在現場採得指紋後送鑑定後，結果與邱東成之指紋相符，而查悉上情。

Top 5 most similar sentences in corpus:
1    105,審易,1055 	 竊盜 	(Score: 0.9833)
詎猶未悔改，復意圖為自己不法之所有，於100年3月18日17時30分前某時許，沿臺中市○○區○○路000號2樓之10林益宏住處陽臺外之鐵管攀爬進入陽臺後開啟未上鎖之窗戶，侵入該房屋內，徒手竊取林益宏所


2    102,審訴,269 	 搶奪 	(Score: 0.9831)
詎仍不知悔改，竟仍意圖為自己不法之所有，基於搶奪之犯意，於101年11月5日中午12時許，騎乘車牌號碼000-000號重型機車，行經桃園市○○○街00號新雅飯店前，見姚德慈獨自步行，認有機可乘，徒手搶


3    105,審易,1268 	 竊盜 	(Score: 0.9829)
詎仍不知悔改，於99年12月27日凌晨3時許之夜間，行經陳幸慧位於臺中市○○區○○路00號住處前，發現該住處1樓落地鋁門窗未固定，認有機可趁，竟基於意圖為自己不法所有之竊盜犯意，徒手扳開該未固定之落地


4    102,訴,1926 	 搶奪 	(Score: 0.9816)
詎猶不知悔改，復意圖為自己不法之所有，基於搶奪之犯意，於102年10月2日晚間10時30分許，在新北市○○區○○路000○0號前，乘行人柯麗美不及防備之際，猝然徒手搶奪柯麗美所持有之皮包及其內財物（內


COS相似計算時間 0.078762 sec





Query: 易佳騰於民國103年12月30日22時53分許，在臺中市○○區○○街000號，因工程款糾紛與李佳樺發生爭執。易佳騰見李佳樺欲進入上址，因而屈肘於李佳樺之領口前，欲攔阻李佳樺，隨即並基於傷害之犯意，徒手推擠李佳樺，並以腳踹李佳樺之腿部，致李佳樺受有左大腿、小腿擦傷、左踝

In [29]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
embedder = SentenceTransformer('bert-base-chinese')
corpus_embeddings = embedder.encode(corpus)
# Then, we perform k-means clustering using sklearn:
from sklearn.cluster import KMeans

specialcls = -1

num_clusters = 20
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

Exception when trying to download https://sbert.net/models/bert-base-chinese.zip. Response 404


In [30]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])
    if (specialtest == corpus[sentence_id]):
        specialcls = cluster_id
        print (cluster_id)
for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster[0].strip()[0:100])
    print("")

16
Cluster  1
詎其仍不知悛悔，於一百年三月七日上午十時五十分許，在臺中市○區○○路與興進路口處，見鑰匙一支插在林孟良所有之車牌號碼ＫＣＴ－二七三號重型機車（上開機車前於一百年二月二十二日二十二時四十分許，在臺中市○

Cluster  2
於九十九年十月六日某時許，在其位於臺南市○區○○街二段七九巷一四號之一之住處房間內，以將第一級毒品海洛因置入針筒內注射之方式，施用海洛因一次。另基於施用第二級毒品甲基安非他命之犯意，於同日某時許，在上

Cluster  3
詎其不知悔改，復基於賭博及意圖營利供給賭博場所、聚眾賭博之犯意，自民國100年1月25日起，提供其位在臺中市○○區○○路230號之公眾得出入之場所，聚集不特定多數人簽選號碼與之賭博財物。其賭法係賭客以

Cluster  4
被告賴文章與其母即告訴人陳張蕉治同住於臺北市○○區○○路○○巷○號1樓，與告訴人陳錦宏為兄弟關係，與告訴人陳怡文則為叔姪關係，為家庭暴力防治法第3條所定之家庭成員關係。陳錦宏、陳怡文因除夕夜返家與陳張

Cluster  5
吳火成於民國99年12月15日下午2時25分許，行經位於臺中市○區○○路一段199號之「行政院衛生署立臺中醫院」(下稱臺中醫院)停車場時，見葉秀華停放在該停車場內之車牌號碼為SZS-048號輕型機車座

Cluster  6
范花春基於反覆實施意圖營利，供給賭博場所、聚眾賭博及賭博之犯意，自民國99年11月底至99年12月初間某日起迄100年4月14日19時30分許為警查獲時止，由范花春提供不知情之其女趙梅櫻所有之臺中縣大

Cluster  7
許其昌係址設臺中市○區○○○路374號「左岸國際美容生活館」(下稱左岸美容館)之負責人，而李耀忠自民國99年3月中旬某日受僱於許其昌、擔任與該美容館之經理，許其昌、李耀忠共同意圖使女子與他人為性交、猥

Cluster  8
張麗娜為張麗莉之妹，其基於公然侮辱之犯意，於民國105年3月21日11時30分許，在不特定多數人得以共見共聞之臺灣臺中地方法院檢察署2樓第14偵查庭外走道，於張麗莉在場之情形下，對張麗莉之女婿徐順承喧

Cluster  9
涂煌輝係原址設嘉義市○區○○路○○○號七樓（現變更為嘉義市○區○○路○段○○○號）之「振揚影音科技有限公司」（下稱振揚公司）之代表人，以提供伴唱歌曲之出租為業。涂煌

In [31]:
class Cls(object):
    def __init__(self):
        self.reason = ""
        self.num = 0

class Cluster_Main(object):
    def __init__(self):
        self.cluster_id = 0
        self.clslist = {}

In [32]:
no = 1
clusterList = {}
show = 0
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if (show < 5):
        print(no, "  "
                  , corpusList[sentence_id].no,"\t"
                  ,  corpusList[sentence_id].reason, "\n")
        if (len(corpusList[sentence_id].Situation.strip()) > 100):
            print(corpusList[sentence_id].Situation.strip()[0:100])
        else:
            print(corpusList[sentence_id].Situation.strip())
        no += 1
        print("")

        show +=1
    if cluster_id in clusterList:
        if corpusList[sentence_id].reason in clusterList[cluster_id]:
            clusterList[cluster_id][corpusList[sentence_id].reason] += 1
        else:
            clusterList[cluster_id][corpusList[sentence_id].reason] = 1
    else:
        clslist = {}
        clslist[corpusList[sentence_id].reason] = 1
        clusterList[cluster_id] = clslist

1    106,易,1093 	 妨害公務 

邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃園分局埔子派出所內（下稱埔子派出所）之值班臺，明知員警徐芳振係依法執行職務之公務員，因不滿員警告以所詢事項應

2    100,中交簡,100 	 公共危險 

林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附近之某工地內，與友人飲用酒類後，已達不能安全駕駛動力交通工具之程度，猶於同日下午6時許，駕駛車牌號碼CX-6

3    100,中交簡,1016 	 公共危險 

朱建鴻於民國100年4月19日晚間9時許起，在臺中市大雅區清泉崗友人住處內飲用啤酒，至同日晚間10時許已達不能安全駕駛動力交通工具之程度，猶於飲畢後騎乘車牌號碼187-BCV號重型機車上路欲返回住處，

4    100,中簡,1001 	 竊盜 

被告陳守仁為高職畢業擔任保全之男子，竟不思以正常手段獲取財物，因一時貪念，以徒手竊取該超商內商品供己食用之犯罪動機、目的、手段、生活狀況、智識程度、所竊得物品價值新臺幣（下同）總共約821元

5    100,中簡,1011 	 賭博 

詎其不知悔改，復基於賭博及意圖營利供給賭博場所、聚眾賭博之犯意，自民國100年1月25日起，提供其位在臺中市○○區○○路230號之公眾得出入之場所，聚集不特定多數人簽選號碼與之賭博財物。其賭法係賭客以



In [33]:
idcnt = 0

for key, value in sorted(clusterList.items()):
    print(key, " ")
    clutotal = sum(value.values())
    print(clutotal)
    for _key, _value in sorted(value.items(), key=lambda x: -x[1]):
        idcnt += _value
        clusterList[key][_key] = value[_key] / clutotal
        print(_key, " ", _value)
        #print(_key, " ", _value, "\nP= ",clusterList[key][_key])
    print(" ")

0  
218
毀損   134
毀棄損壞   11
搶奪   10
妨害自由   10
侵占   8
殺人未遂   6
傷害   6
妨害名譽   6
過失傷害   5
強盜   4
妨害公務   4
偽造文書   4
竊盜   3
詐欺   2
妨害風化   2
公共危險   1
妨害性自主   1
槍砲彈藥刀械   1
 
1  
61
毒品危害   53
強盜   3
殺人未遂   3
偽證   1
藥事法   1
 
2  
156
侵占   79
偽造文書   29
毀損   9
詐欺   9
賭博   4
妨害風化   4
著作權法   4
過失致死   4
妨害名譽   3
醫師法   3
竊盜   2
妨害公務   2
公共危險   2
妨害自由   1
貪污   1
 
3  
283
傷害   127
殺人未遂   58
毀損   40
妨害名譽   32
妨害自由   14
殺人   4
毀棄損壞   4
妨害公務   2
強盜   1
搶奪   1
 
4  
212
妨害公務   70
公共危險   38
妨害自由   24
毀損   24
搶奪   13
過失傷害   11
傷害   8
妨害名譽   6
偽造文書   6
殺人未遂   4
竊盜   2
妨害性自主   2
妨害風化   1
毀棄損壞   1
侵占   1
強盜   1
 
5  
286
毀損   78
妨害自由   67
妨害名譽   34
侵占   20
偽造文書   18
毀棄損壞   12
殺人未遂   9
詐欺   9
竊盜   7
妨害風化   6
強盜   6
傷害   6
公共危險   4
妨害公務   3
搶奪   2
妨害性自主   2
賭博   1
著作權法   1
貪污   1
 
6  
162
妨害風化   158
妨害性自主   2
賭博   1
著作權法   1
 
7  
124
妨害名譽   101
妨害自由   12
詐欺   3
妨害風化   2
著作權法   2
偽造文書   1
傷害   1
強盜   1
殺人   1
 
8  
112
著作權法   112
 
9  
207
侵占   43
偽造文書   38
妨害自由   36
妨害名譽   17
詐欺   13
強盜   10
殺人未遂   9
傷害   9
殺人 

In [34]:
from scipy import stats
import pandas as pd

def Entropy(labels):
    # 计算概率分布
    probs = pd.Series(labels).value_counts() / len(labels)
    # 计算底数为base的熵
    en = stats.entropy(probs)
    return en

In [35]:
type(clusterList)

dict

In [36]:
score = 0
clucnt = 0
clusterEntropy = {}
for key, value in sorted(clusterList.items()):
    clusterEntropy[key] = Entropy(value)
    score += clusterEntropy[key]
    clucnt += 1

print("Avg Entropy:\t", score/clucnt, "\n")

for key, value in sorted(clusterEntropy.items()):
    print("Clu", key, "Entropy:\t", value)

Avg Entropy:	 1.7003603402193028 

Clu 0 Entropy:	 2.187032906771011
Clu 1 Entropy:	 1.0549201679861442
Clu 2 Entropy:	 1.8413903748459728
Clu 3 Entropy:	 2.0253262207700677
Clu 4 Entropy:	 2.166084939249829
Clu 5 Entropy:	 2.4515830605217857
Clu 6 Entropy:	 1.0397207708399179
Clu 7 Entropy:	 1.4270610433807247
Clu 8 Entropy:	 0.0
Clu 9 Entropy:	 2.476247132983033
Clu 10 Entropy:	 1.3321790402101223
Clu 11 Entropy:	 2.3439329813080194
Clu 12 Entropy:	 1.2275294114572126
Clu 13 Entropy:	 2.4503962829577546
Clu 14 Entropy:	 1.3296613488547582
Clu 15 Entropy:	 1.8343719702816237
Clu 16 Entropy:	 1.9792045174343245
Clu 17 Entropy:	 2.252728336819822
Clu 18 Entropy:	 1.8946891171539901
Clu 19 Entropy:	 0.6931471805599453
